In [73]:
from fairpair import *

In [74]:
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [75]:
sampler = RandomSampling(G, warn=False)
sampler.apply(iter=100, k=1)

In [76]:
ranker = RankRecovery(G)
ranking, other_nodes = ranker.apply(rank_using=davidScore)

In [77]:
ranker._print_with_score(ranking)

7 0.8716839026684216 55.023809523809504
11 0.5661819365713622 40.511904761904745
9 0.8772139476639081 35.61666666666666
8 0.902759297747564 31.6
6 0.37369173125674615 20.58333333333333
12 0.33122197623272615 11.983333333333327
1 0.5099782834803557 10.55476190476191
0 0.46282081472451864 5.054761904761911
14 0.5059497507929962 -3.4547619047619094
10 0.42759305169560163 -7.050000000000004
13 0.27318059368024644 -20.050000000000004
4 0.15448313651673723 -31.08333333333333
3 0.2208263190211276 -41.35714285714286
5 0.38040555177875346 -45.91666666666666
2 0.11219577698943334 -62.01666666666668
